In [196]:
import pandas as pd
import re
from openpyxl import load_workbook

In [3]:
pd.set_option('display.max_rows', 100)

In [4]:
# partial files because provinces and locals must be added as names - their codes are integers.
# and districts as codes, their codes are municipal demarcation board, and their names exclude their code.
census2011prov = pd.read_csv("census-2011-2016-boundaries-pr-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011localmun = pd.read_csv("census-2011-2016-boundaries-mn-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011districtmun = pd.read_csv("census-2011-2016-boundaries-dc-age-population.csv", skiprows=9, dtype={"Age in completed years": str})
census2011 = pd.concat([census2011prov, census2011districtmun, census2011localmun])

In [5]:
census2011 = census2011.drop(columns=["Summation Options", "Unnamed: 4"])
census2011.columns = ["Geography", "Age", "Count"]
census2011 = census2011[census2011["Geography"] != "Total"]
census2011 = census2011[census2011["Age"] != "Total"]
census2011 = census2011.dropna()

In [6]:
commsurv2016prov = pd.read_csv("community-survey-2016-pr-age-population.csv", skiprows=9, dtype={"Age": str})
commsurv2016mun = pd.read_csv("community-survey-2016-dc-mn-age-population.csv", skiprows=9, dtype={"Age": str})
commsurv2016 = pd.concat([commsurv2016prov, commsurv2016mun])

In [7]:
commsurv2016 = commsurv2016.drop(columns=["Summation Options", "Unnamed: 4"])
commsurv2016 = commsurv2016[["Geography hierarchy 2016", "Age", "Count"]]
commsurv2016.columns = ["Geography", "Age", "Count"]
commsurv2016 = commsurv2016[commsurv2016["Geography"] != "Total"]
commsurv2016 = commsurv2016[commsurv2016["Age"] != "Total"]
commsurv2016 = commsurv2016.dropna()

In [8]:
commsurv2016["Geography"] = commsurv2016["Geography"].apply(lambda cell: re.sub(" *:.+", "", cell))
commsurv2016 = commsurv2016.drop_duplicates(["Geography", "Age"])

In [9]:
census2011["Geography"] = census2011["Geography"].apply(lambda cell: re.sub(" *:.+", "", cell))
census2011 = census2011.drop_duplicates(["Geography", "Age"])

In [146]:
pop2011_2016 = census2011.merge(commsurv2016, how="right", on=["Geography", "Age"], suffixes=("_2011", "_2016"))
replacements = [
    ("Eastern Cape", "EC"),
    ("Northern Cape", "NC"),
    ("Western Cape", "WC"),
    ("Gauteng", "GT"),
    ("Limpopo", "LIM"),
    ("Mpumalanga", "MP"),
    ("KwaZulu-Natal", "KZN"),
    ("Free State", "FS"),
    ("North West", "NW"),
]
for (prov, code) in replacements:
    pop2011_2016["Geography"].replace(prov, code, inplace=True)

In [147]:
bins = [0, 15, 25, 35, 45, 55, 65, float("inf")]
labels = ["0-14", "15-24", "25-34", "35-44", "45-54", "55-64", "65+"]
pop2011_2016["Age group"] = pd.cut(pop2011_2016.Age.astype(int), bins=bins, labels=labels, right=False)

In [182]:
pop2011_2016_age_grouped = pop2011_2016.groupby(["Geography", "Age group"])
pop2011_2016_age_grouped = pop2011_2016_age_grouped.sum(numeric_only=True)

# agegrp_abs_change represents the absolute change in population of a given age group in a given geography.
pop2011_2016_age_grouped["agegrp_abs_change"] = pop2011_2016_age_grouped.apply(lambda row: row["Count_2016"] - row.Count_2011, axis = 1)

# 2011_agegrp_proportion represents the proportion of an age group of the total population for a geography in 2011
pop2011_2016_age_grouped["2011_agegrp_proportion"] = pop2011_2016_age_grouped['Count_2011'] / pop2011_2016_age_grouped.groupby('Geography')['Count_2011'].transform('sum')
# 2016_agegrp_proportion represents the proportion of an age group of the total population for a geography in 2016
pop2011_2016_age_grouped["2016_agegrp_proportion"] = pop2011_2016_age_grouped['Count_2016'] / pop2011_2016_age_grouped.groupby('Geography')['Count_2016'].transform('sum')

# agegrp_proportion_change represents the change in the proportion an age group represents
# as a total of the population in that geography. e.g. if the age group 0-14 represented 
# 20% of the population in 2011 and 25% of the population in 2016, then agegrp_proportion_change 
# for age group 0-14 would be 5 in that geography.
pop2011_2016_age_grouped["agegrp_proportion_change"] = pop2011_2016_age_grouped["2016_agegrp_proportion"] - pop2011_2016_age_grouped["2011_agegrp_proportion"]

# agegrp_proportion_change_pct represents the percentage by which the proportion of each 
# age group in the geography changed. Since the absolute change in proportion is likely to 
# be proportional to the original proportion, this variable is more useful to highlight changes
# that are significant relative to the original population.
pop2011_2016_age_grouped["agegrp_proportion_change_pct"] = pop2011_2016_age_grouped["agegrp_proportion_change"] / pop2011_2016_age_grouped["2011_agegrp_proportion"]

# agegrp_magnitude_pct_change represents the percentage that an age group changed. 
# i.e. if the age group 0-14 had 100,000 people, and now it has 120,0000 people,
# its agegrp_magnitude_pct_change value would be 0.20, i.e. the 0-14 age group grew by 20%.
pop2011_2016_age_grouped["agegrp_magnitude_pct_change"] = pop2011_2016_age_grouped["agegrp_abs_change"] / pop2011_2016_age_grouped["Count_2011"]
pop2011_2016_age_grouped

Count_2011     Count_2016  agegrp_abs_change  \
Geography Age group                                                   
BUF       0-14       207251.47372  263537.435480       56285.961760   
          15-24      149446.23804  140357.932284       -9088.305756   
          25-34      131911.14793  147875.633966       15964.486036   
          35-44      103506.20062  106830.650515        3324.449895   
          45-54       86188.95151   83120.916322       -3068.035188   
...                           ...            ...                ...   
WC053     25-34        6955.18850    8521.849012        1566.660512   
          35-44        6561.13090    6239.627996        -321.502904   
          45-54        5511.05895    5251.883376        -259.175574   
          55-64        3701.53360    3380.797352        -320.736248   
          65+          2934.36586    3549.635434         615.269574   

                     2011_agegrp_proportion  2016_agegrp_proportion  \
Geography Age group                                                   
BUF       0-14                     0.265359                0.315615   
          15-24                    0.191347                0.168094   
          25-34                    0.168896                0.177097   
          35-44                    0.132527                0.127941   
          45-54                    0.110354                0.099546   
...                                     ...                     ...   
WC053     25-34                    0.140265                0.166833   
          35-44                    0.132318                0.122154   
          45-54                    0.111141                0.102817   
          55-64                    0.074649                0.066186   
          65+                      0.059177                0.069492   

                     agegrp_proportion_change  agegrp_proportion_change_pct  \
Geography Age group                                                           
BUF       0-14                       0.050255                      0.189386   
          15-24                     -0.023253                     -0.121523   
          25-34                      0.008202                      0.048560   
          35-44                     -0.004585                     -0.034599   
          45-54                     -0.010808                     -0.097937   
...                                       ...                           ...   
WC053     25-34                      0.026568                      0.189415   
          35-44                     -0.010164                     -0.076816   
          45-54                     -0.008325                     -0.074900   
          55-64                     -0.008462                     -0.113363   
          65+                        0.010314                      0.174297   

                     agegrp_magnitude_pct_change  
Geography Age group                               
BUF       0-14                          0.271583  
          15-24                        -0.060813  
          25-34                         0.121025  
          35-44                         0.032118  
          45-54                        -0.035597  
...                                          ...  
WC053     25-34                         0.225251  
          35-44                        -0.049001  
          45-54                        -0.047028  
          55-64                        -0.086650  
          65+                           0.209677  

[1862 rows x 8 columns]

In [149]:
pop2011_2016_geo_grouped = pop2011_2016.groupby(["Geography"]).sum(numeric_only=True)
pop2011_2016_geo_grouped["absolute_change"] = pop2011_2016_geo_grouped.apply(lambda row: row["Count_2016"] - row.Count_2011, axis = 1)

def calculate_percentage(row):
    if row.Count_2016 == 0:
        return 0
    if row.Count_2011 == 0:
        return None
    return row["absolute_change"] / row.Count_2011
        
pop2011_2016_geo_grouped["percentage_change"] = pop2011_2016_geo_grouped.apply(calculate_percentage, axis = 1)

In [150]:
pop2011_geo_grouped = pop2011_2016_geo_grouped[["Count_2011"]].copy()
pop2011_geo_grouped.columns = ["Count"]
pop2011_geo_grouped["category"] = "Total population"
pop2011_geo_grouped.to_excel("pop2011_geo_grouped.xlsx")

In [151]:
pop2016_geo_grouped = pop2011_2016_geo_grouped[["Count_2016"]].copy()
pop2016_geo_grouped.columns = ["Count"]
pop2016_geo_grouped["category"] = "Total population"
pop2016_geo_grouped.to_excel("pop2016_geo_grouped.xlsx")

In [152]:
pop2011_2016_abs_geo_grouped = pop2011_2016_geo_grouped[["absolute_change"]].copy()
pop2011_2016_abs_geo_grouped.columns = ["Count"]
pop2011_2016_abs_geo_grouped["category"] = "Population change (absolute)"
pop2011_2016_abs_geo_grouped.to_excel("pop2011_2016_abs_geo_grouped.xlsx")

In [153]:
pop2011_2016_pct_geo_grouped = pop2011_2016_geo_grouped[["percentage_change"]].copy()
pop2011_2016_pct_geo_grouped.columns = ["Count"]
pop2011_2016_pct_geo_grouped["category"] = "Population change (%)"
pop2011_2016_pct_geo_grouped.to_excel("pop2011_2016_pct_geo_grouped.xlsx")

In [154]:
pop2011_geo_grouped_year_var = pop2011_geo_grouped.copy()
pop2011_geo_grouped_year_var["category"] = "2011"
pop2011_geo_grouped_year_var.columns = ["Count", "year"]

In [155]:
pop2016_geo_grouped_year_var = pop2016_geo_grouped.copy()
pop2016_geo_grouped_year_var["category"] = "2016"
pop2016_geo_grouped_year_var.columns = ["Count", "year"]

In [156]:
pop2011_2016_geo_grouped_year_var = pd.concat([pop2011_geo_grouped_year_var, pop2016_geo_grouped_year_var])
pop2011_2016_geo_grouped_year_var.to_excel("pop2011_2016_geo_grouped_year_var.xlsx")

In [201]:
pop2011_age_grouped = pop2011_2016_age_grouped[["Count_2011"]].copy()
pop2011_age_grouped["year"] = "2011"
pop2011_age_grouped.columns = ["Count", "year"]
pop2016_age_grouped = pop2011_2016_age_grouped[["Count_2016"]].copy()
pop2016_age_grouped["year"] = "2016"
pop2016_age_grouped.columns = ["Count", "year"]
filename = "pop2011_2016_age_grouped_year_var.xlsx"
pd.concat([pop2011_age_grouped, pop2016_age_grouped]).reset_index().to_excel(filename, index=False)
wb = load_workbook(filename)
ws = wb.create_sheet(title="Documentation")
ws["A1"] = "Absolute population for 2011 and 2016."
wb.save(filename)

In [203]:
pop2011_2016_agegrp_abs_change = pop2011_2016_age_grouped[["agegrp_abs_change"]].copy()
pop2011_2016_agegrp_abs_change.columns = ["Count"]
filename = "pop2011_2016_agegrp_abs_change.xlsx"
pop2011_2016_agegrp_abs_change.reset_index().to_excel(filename, index=False)
wb = load_workbook(filename)
ws = wb.create_sheet(title="Documentation")
ws["A1"] = "Absolute population change between 2011 and 2016 for each age group in each geography."
wb.save(filename)

In [206]:
pop2011_2016_agegrp_proportion_change = pop2011_2016_age_grouped[["agegrp_proportion_change"]].copy()
pop2011_2016_agegrp_proportion_change.columns = ["Count"]
filename = "pop2011_2016_agegrp_proportion_change.xlsx"
pop2011_2016_agegrp_proportion_change.reset_index().to_excel(filename, index=False)
wb = load_workbook(filename)
ws = wb.create_sheet(title="Documentation")
ws["A1"] = (
    "This represents the change in the proportion an age group represents\n"
    "as a total of the population in that geography. e.g. if the age group 0-14 represented \n"
    "20% of the population in 2011 and 25% of the population in 2016, then this value"
    "for age group 0-14 would be 5 in that geography."
)
wb.save(filename)

In [207]:
pop2011_2016_agegrp_proportion_change_pct = pop2011_2016_age_grouped[["agegrp_proportion_change_pct"]].copy()
pop2011_2016_agegrp_proportion_change_pct.columns = ["Count"]
filename = "pop2011_2016_agegrp_proportion_change_pct.xlsx"
pop2011_2016_agegrp_proportion_change_pct.reset_index().to_excel(filename, index=False)
wb = load_workbook(filename)
ws = wb.create_sheet(title="Documentation")
ws["A1"] = (
    "This represents the percentage by which the proportion of each\n"
    "age group in the geography changed. Since the absolute change in proportion is likely to\n"
    "be proportional to the original population proportion of the age group, this variable is more useful to highlight changes\n"
    "that are significant relative to the original population."
)
wb.save(filename)

In [208]:
pop2011_2016_agegrp_magnitude_pct_change = pop2011_2016_age_grouped[["agegrp_magnitude_pct_change"]].copy()
pop2011_2016_agegrp_magnitude_pct_change.columns = ["Count"]
filename = "pop2011_2016_agegrp_magnitude_pct_change.xlsx"
pop2011_2016_agegrp_magnitude_pct_change.reset_index().to_excel(filename, index=False)
wb = load_workbook(filename)
ws = wb.create_sheet(title="Documentation")
ws["A1"] = (
    "This represents the percentage that an age group population changed between 2011 and 2016.\n"
    "i.e. if the age group 0-14 had 100,000 people, and now it has 120,0000 people,\n"
    "its agegrp_magnitude_pct_change value would be 0.20, i.e. the 0-14 age group grew by 20%."
)
wb.save(filename)